# GPT-3 Embeddings
**An updated approach to semantic search.**

The embedding is an **information dense representation of the semantic meaning of a piece of text**.
Each embedding is a vector of floating point numbers, such that the **distance between two embeddings in the vector space is correlated with semantic similarity between two inputs** in the original format.
For example, if two texts are similar, then their vector representations should also be similar.

Use cases:

- Text Similarity
- Semantic Search
- Classification
- Clustering

1. **Similarity embeddings** : These models are good at capturing semantic similarity between two or more pieces of text.
2. **Text search embeddings**: These models help measure whether long documents are relevant to a short search query. There are two types: one for *embedding the documents* to be retrieved, and one for **embedding the search query**.

In [1]:
# Import required libraries

import os
import openai, numpy as np, pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
# API key stored as an environment variable

openai.api_key = os.environ["OPENAI_API_KEY"]
# print(openai.api_key) # You don't wanna do that.

In [3]:
# Text Similarity: captures semantic similarity between pieces of text.

document = ["eating food", "I am hungry", "I am traveling", "exploring new places"]
response = openai.Embedding.create(
    input=document,
    engine="text-similarity-curie-001"
)

In [6]:
# Interrogate response

print(*[f"{type(r)} | {len(r)}" for r in [response["data"], response["data"][0]]], sep='\n')
print(response["data"][0].keys())
print(response["data"][0]["embedding"][:20])

<class 'list'> | 4
<class 'openai.openai_object.OpenAIObject'> | 3
dict_keys(['object', 'index', 'embedding'])
[-0.0019766087643802166, 0.0014332111459225416, -0.015559284016489983, 0.011673991568386555, -0.01459022518247366, 0.009178890846669674, -0.0053796363063156605, -0.011148707009851933, 0.010596252977848053, 0.02966950833797455, 0.0068060546182096004, -0.015504944138228893, -0.007222659420222044, 0.011357009410858154, 0.016854381188750267, 0.014372865669429302, -0.019580425694584846, -0.026680821552872658, 0.01044229045510292, 0.001698117470368743]


In [7]:
# Embeddings (vector representation) for the contents of `document`

embedding_a = response['data'][0]['embedding']  # eating food
embedding_b = response['data'][1]['embedding']  # I am hungry
embedding_c = response['data'][2]['embedding']  # I am traveling
embedding_d = response['data'][3]['embedding']  # exploring new places

In [8]:
# Compare embeddings (using dot products)

print(np.dot(embedding_a, embedding_b))   # eating food vs I am hungry
print(np.dot(embedding_a, embedding_c))   # eating food vs I am traveling
print(np.dot(embedding_c, embedding_d))   # I am traveling vs exploring new places

0.8482169927861086
0.7816395422937359
0.8348286175991912


In [9]:
# https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
# Precomputed embeddings

datafile_path = "https://cdn.openai.com/API/examples/data/fine_food_reviews_with_embeddings_1k.csv"
df = pd.read_csv(datafile_path)
df.head()

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,babbage_similarity,babbage_search
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,51,"[-0.01274053193628788, 0.010849879123270512, -...","[-0.01880764216184616, 0.019457539543509483, -..."
1,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,35,"[-0.024154752492904663, 0.0024838377721607685,...","[-0.03571609780192375, 0.010356518439948559, -..."
2,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....",277,"[0.0032693513203412294, 0.017815979197621346, ...","[-0.010433986783027649, 0.024620095267891884, ..."
3,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,246,"[-0.03584608808159828, 0.03424076735973358, -0...","[-0.040209852159023285, 0.03804996609687805, -..."
4,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,87,"[0.005218076519668102, 0.018165964633226395, -...","[0.010450801812112331, 0.022801749408245087, -..."


In [11]:
# Interrogate datafile

# Check data type
type(df.loc[0]["babbage_search"])  # str

# Convert string with square bracket to array using eval expression then subsequently to numpy array
df["babbage_search"] = df.babbage_search.apply(eval).apply(np.array)

# Check updated data type
type(df.loc[0]["babbage_search"])  # numpy.ndarray

numpy.ndarray

## Semantic Search

In [14]:
# Search through the reviews for a specific product

def search_reviews(dataframe, search_query, n=3):
    """Takes an embedding `x` and calculates the cosine similarity between `x` and the embedding of `search_query`.
    """
    
    # Calculate embedding of `search_query`
    embedding = get_embedding(
        search_query,
        engine="text-search-babbage-query-001"
    )
    
    # Store a similarity (cosine) score for each embedding comparison under the `similarity` column
    df["similarities"] = dataframe.babbage_search.apply(lambda x: cosine_similarity(x, embedding))
    
    # Return top 3 similarities
    top_n = dataframe.sort_values("similarities", ascending=False).head(n)
    return top_n

In [15]:
# Run sample semantic search query

result = search_reviews(dataframe=df, search_query="delicious beans", n=3)
result["combined"].to_list()

['Title: Fantastic Instant Refried beans; Content: Fantastic Instant Refried Beans have been a staple for my family now for nearly 20 years.  All 7 of us love it and my grown kids are passing on the tradition.',
 'Title: Jamaican Blue beans; Content: Excellent coffee bean for roasting. Our family just purchased another 5 pounds for more roasting. Plenty of flavor and mild on acidity when roasted to a dark brown bean and before any oil appears on the bean itself (455F @ 17 minutes).',
 "Title: Delicious!; Content: I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is traying different seasoning and she likes it very much.<br />Thank you Amazon for having it because now I can't find it in stores, I like to have this 12 boxes because I can made it whenever my family want it."]

## Classification

In [16]:
# Import required libraries

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [21]:
# Split dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    list(df.babbage_similarity.values),
    df.Score,
    test_size=0.2,
    random_state=42
)

In [22]:
# Fit the dataset (embeddings) in the model (Random Forest)

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

ValueError: could not convert string to float: '[-0.009171920828521252, 0.01047194842249155, -0.027058511972427368, -0.012345781549811363, 0.013493392616510391, 0.011063685640692711, -0.003324036719277501, 0.00863397866487503, 0.011180239729583263, -0.025408821180462837, 0.004505269229412079, -0.0003914092667400837, 0.0058366768062114716, 0.003368865232914686, -0.008512941189110279, -0.011951291002333164, -0.019150756299495697, -0.021320456638932228, 0.02089010365307331, 0.07197671383619308, 0.12057086080312729, 0.017052778974175453, -0.0011509729083627462, 0.006988770328462124, 0.021804604679346085, -0.006818422116339207, -0.038552552461624146, 0.015752751380205154, -0.030268235132098198, 0.007791201584041119, 0.0051328688859939575, -0.0017673654947429895, 0.01954524591565132, -0.030178578570485115, 0.00044996655196882784, 0.006697385106235743, 0.012632684782147408, -0.00910467840731144, 0.019670765846967697, -0.034320734441280365, -0.010839542374014854, -0.010516777634620667, -0.0132